# Eccentricity Coefficients for the Circumbinary Gravitational Potential

In this notebook, we compute the eccentricity coefficients that appear in the expansion of the gravitational potential due to a binary point mass system executing an eccentric orbit.

The gravitational potential has the form:

$$
\Phi = -2\sum_{l,m,\mu}Q_lW_{l,m}W_{l,m'}d^l_{\mu,m}(i)r^{-l-1}r_{12}^l\cos(m\phi - \mu\phi') 
$$

In this case, one has $r_{12}$ as the binary separation and $\phi'$ as the true anomaly of the binary. In order to get the gravitational potential in the form of

$$
\Phi = \sum_{m,N} \Phi_{m,N}\cos(m\phi - NM) 
$$

where $M = \Omega_b t $ is the mean anomaly of the binary, one must find the coefficients $C^l_{\mu n}$ such that:

$$
r_{12}^l\cos(m\phi - \mu\phi') = \sum_{n} C^l_{\mu,n}\cos(m\phi - (\mu + n)M).
$$

This then allows one to write the potential as

$$
\Phi_{m,N} = \sum_{\mu,n} \Phi_{m,\mu,n}\delta_{\mu + n,N}.
$$

Clearly, in order to do this we must solve both the binary separation and the true anomaly in terms of the mean anomaly which in turn requires solving the Kepler equation.

## Solving the Kepler Equation for the Eccentric Anomaly

In this section, we will solve the Kepler Equation

$$
M = E - e\sin E
$$

for the eccentric anomaly E. We wish to get an approximate solution up to $\mathcal{O}(e^{10})$, so we use Taylor series to write the equation as a polyomial and solve for each of the coefficients. That is, we introduce:

$$
E = E_0 + E_ne^n\\
\Rightarrow \sin(E) = \sin(E_0) + E_ne^n\cos(E_0) - \frac{e^ne^mE_mE_m}{2!}\sin(E_0) + ...  
$$

We validate the first four coefficients against the solution quoted in ML2015 as

$$
E = M + e\sin M + \frac{e^2}{2}\sin 2M + \frac{e^3}{8}(3\sin 3M - \sin M) + \frac{e^4}{6}(2 \sin 4M - \sin 2M)
$$

In [1]:
import sympy as sp
import numpy as np

## Solve for the Eccentric Anomaly
## M = E - e*sinE

M,e = sp.symbols("M e")
E = M
N = 5
for n in range(1,N):
  dE = sp.Symbol("dE")
  kepler_eqn = M - (E + dE*(e**n)) + e*sp.sin(E+dE*(e**n))
  dE_soln = sp.solve(kepler_eqn.taylor_term(n,e),dE)
  E += (e**n)*sp.fourier_series(dE_soln[0],(M,0,2*sp.pi)).truncate(N)

E_sol = E
E_ML2015 = M + e*sp.sin(M) + e*e*sp.sin(2*M)/2 + e*e*e*(3*sp.sin(3*M) - sp.sin(M))/8 + e*e*e*e*(2*sp.sin(4*M) - sp.sin(2*M))/6
print(E_sol)
for n in range(5):
  if sp.simplify(E_sol.taylor_term(n,e) - E_ML2015.taylor_term(n,e)) != 0:
    print("term of order ",n," does not agree")
    break
  if n == 4:
    print("All terms agree with ML2015")

M + e**4*(-sin(2*M)/6 + sin(4*M)/3) + e**3*(-sin(M)/8 + 3*sin(3*M)/8) + e**2*sin(2*M)/2 + e*sin(M)
All terms agree with ML2015


## Compute the Binary Separation

The binary separation is given in terms of the eccentric anomaly as:

$$
r_{12} = 1 - e\cos E
$$

In [2]:
## solve for the radius
## r = 1 - e*cosE

r12 = 1 - e*sp.cos(E_sol)
r12_sol = 1
for n in range(1,N):
  term = sp.simplify(r12.taylor_term(n,e))
  r12_sol += term

print(r12_sol)
r12_ML2015 = 1 - e*sp.cos(M) + e*e*(1 - sp.cos(2*M))/2 + 3*e*e*e*(sp.cos(M) - sp.cos(3*M))/8 + e*e*e*e*(sp.cos(2*M) - sp.cos(4*M))/3

for n in range(5):
  if sp.simplify(r12_sol.taylor_term(n,e) - r12_ML2015.taylor_term(n,e)) != 0:
    print("term of order ",n," does not agree")
    break
  if n == 4:
    print("All terms agree with ML2015")

2*e**4*sin(M)*sin(3*M)/3 + 3*e**3*sin(M)*sin(2*M)/4 + e**2*sin(M)**2 - e*cos(M) + 1
All terms agree with ML2015


## Compute the True Anomaly $\phi'$

The true anomaly is given in terms of the binary separation as

$$
\phi' = \int^MdM \frac{\sqrt{1 - e^2}}{r_{12}^2}
$$

In [3]:
## solve for the true anomaly
## P = int_0^M dM sqrt(1-e^2)/(r12^2)

integrand = sp.sqrt(1 - e*e)/(r12_sol**2)
integrand_true = sum([sp.simplify(integrand.taylor_term(n,e)) for n in range(N)])
integral = sp.integrate(integrand_true,M)
P_sol = sum([sp.simplify(integral.taylor_term(n,e)) for n in range(N)])

P_ML2015 = M + 2*e*sp.sin(M) + 5*e*e*sp.sin(2*M)/4 + e*e*e*(13*sp.sin(3*M)/12 - sp.sin(M)/4) + e*e*e*e*(103*sp.sin(4*M)/96 - 11*sp.sin(2*M)/24)
print(P_sol)
for n in range(5):
  if sp.simplify(P_sol.taylor_term(n,e) - P_ML2015.taylor_term(n,e)) != 0:
    print("term of order ", n, " does not agree")
    break
  if n == 4:
    print("All terms agree with ML2015")

M + e**4*(-11*sin(2*M) + 103*sin(4*M)/4)/24 + e**3*(-3*sin(M) + 13*sin(3*M))/12 + 5*e**2*sin(2*M)/4 + 2*e*sin(M)
All terms agree with ML2015


## Compute the Eccentric Coefficients Numericaly

The eccentric coefficients are given by a transformation of the point

$$
r^l_{12}\cos(m\phi - \mu \phi') = \sum_{n}C^l_{\mu,n}\cos\left(m\phi - (\mu+n)M\right)
$$

Thus one can invert this equation using an inverse fourier transform and get

$$
C^l_{\mu n} = \frac{1}{2\pi^2}\int_0^{2\pi}dM\int_0^{2\pi}d\phi r_{12}^l(M)\cos(m\phi - \mu \phi'(M))\cos\left(m\phi - (\mu+n)M\right)
$$


In [4]:
## Compute the Eccentric Coefficients

l = sp.Symbol("l", integer = True)
mu = sp.Symbol("mu", integer = True)
phi = sp.Symbol("phi")
C = []

integrand = (r12_sol**l)*sp.cos(mu*P_sol)*sp.cos((mu+1)*M)
true_integrand = integrand.taylor_term(1,e)
print(true_integrand)
Clmu1 = sp.integrate(true_integrand,(M,0,sp.pi))
print(Clmu1)

e*(-l*cos(M)*cos(M*mu)*cos(M*(mu + 1)) - 2*mu*sin(M)*sin(M*mu)*cos(M*(mu + 1)))
e*(l*Piecewise((-pi/8, Eq(mu, -1)), (pi/8, Eq(mu, 1)), (0, True)) - l*Piecewise((3*pi/8, Eq(mu, -1)), (pi/2, Eq(mu, 0)), (3*pi/8, Eq(mu, 1)), (pi*mu**3/(4*mu**3 - 4*mu) - pi*mu/(4*mu**3 - 4*mu), True)) - 2*mu*Piecewise((-pi/8, Eq(mu, -1)), (pi/8, Eq(mu, 1)), (0, True)) + 2*mu*Piecewise((3*pi/8, Eq(mu, -1)), (0, Eq(mu, 0)), (3*pi/8, Eq(mu, 1)), (pi*mu**3/(4*mu**3 - 4*mu) - pi*mu/(4*mu**3 - 4*mu), True)))
